<a href="https://colab.research.google.com/github/Dr-Future/CSC413-project/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CURRENT PREGRESS AND NOTES:

It takes 10 sec to run 1 epoch on colab

After 100 epochs of training the model performs much better. Problems is, in the area where it's supposed to be completely black or white, the model outputs opposite color pixel.

Maybe try Adam rather than SGD

I forgot to save the work after training for 100 epochs

# Import and Download

In [85]:
import os
import math
import time

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import numpy.random as npr
import torch
from torchvision.transforms.functional import InterpolationMode
from torchvision.transforms import Compose, Resize, ToTensor, ToPILImage
import torchvision.transforms as T
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [2]:
# 67M
!curl --remote-name -L https://www.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/BSR/BSR_bsds500.tgz
!tar xzf BSR_bsds500.tgz
!rm BSR_bsds500.tgz
!mv BSR/BSDS500/data/images/* BSR/BSDS500
!rm -r BSR/bench
!rm -r BSR/documentation
!rm -r BSR/BSDS500/data

# Move all samples in test folder to train folder
# So that we have 400 training examples
# %mv BSR/BSDS500/data/images/test/*.jpg BSR/BSDS500/data/images/train

# 428M
!curl --remote-name -L https://image-net.org/data/ILSVRC/2017/ILSVRC2017_DET_test_new.tar.gz
!tar xzf ILSVRC2017_DET_test_new.tar.gz
!rm ILSVRC2017_DET_test_new.tar.gz
!mv ILSVRC/Data/DET/test/*.JPEG ILSVRC
!rm -r ILSVRC/Data
!rm -r ILSVRC/ImageSets

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 67.4M  100 67.4M    0     0  12.2M      0  0:00:05  0:00:05 --:--:-- 14.5M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  427M  100  427M    0     0  4029k      0  0:01:48  0:01:48 --:--:-- 16.5M


# Utilities

In [3]:
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

In [4]:
class BSDS500Dataset(torch.utils.data.Dataset):
    def __init__(self, split, scale, resize):
        self.root = "BSR/BSDS500"
        self.split = split # train / test / val
        self.scale = scale
        self.resize = resize
        self.low_res_size = self.resize // self.scale
        self.high_res_size = self.low_res_size * self.scale
        self.file_names = [os.path.join(self.root, self.split, file_name) for file_name in os.listdir(os.path.join(self.root, self.split)) if file_name.endswith(".jpg")]

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, index):
        # Convert to YUV and split the channels
        image = Image.open(self.file_names[index]).convert("YCbCr")
        y, u, v = image.split()

        # Rescale images to square
        low_composed = Compose([
            Resize([self.low_res_size, self.low_res_size], interpolation=InterpolationMode.BICUBIC),
            ToTensor()
        ])

        high_composed = Compose([
            Resize([self.high_res_size, self.high_res_size], interpolation=InterpolationMode.BICUBIC),
            ToTensor()
        ])

        return { "y_low": low_composed(y).cuda(), "u_low": low_composed(u).cuda(), "v_low": low_composed(v).cuda(),
               "y_high": high_composed(y).cuda(), "u_high": high_composed(u).cuda(), "v_high": high_composed(v).cuda(),
               "high": high_composed(image).cuda() }


In [5]:
class ILSVRC2017Dataset(torch.utils.data.Dataset):
    def __init__(self, scale: int, resize: int):
        self.root = "ILSVRC"
        self.scale = scale
        self.resize = resize
        self.low_res_size = self.resize // self.scale
        self.high_res_size = self.low_res_size * self.scale
        self.file_names = [os.path.join(self.root, file_name) for file_name in os.listdir(self.root) if file_name.endswith(".JPEG")]

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, index):
        # Convert to YUV and split the channels
        image = Image.open(self.file_names[index]).convert("YCbCr")
        y, u, v = image.split()

        # Rescale images to square
        low_composed = Compose([
            Resize([self.low_res_size, self.low_res_size], interpolation=InterpolationMode.BICUBIC),
            ToTensor()
        ])

        high_composed = Compose([
            Resize([self.high_res_size, self.high_res_size], interpolation=InterpolationMode.BICUBIC),
            ToTensor()
        ])

        return { "y_low": low_composed(y).cuda(), "u_low": low_composed(u).cuda(), "v_low": low_composed(v).cuda(),
               "y_high": high_composed(y).cuda(), "u_high": high_composed(u).cuda(), "v_high": high_composed(v).cuda(),
               "high": high_composed(image).cuda() }

In [99]:
# Calculate PSNR between two images (tensor)
def psnr(img_1, img_2):
    mse = torch.mean((img_1 - img_2) ** 2)
    if(mse == 0):
        return 100
    max_pixel = 1
    psnr = 20 * math.log10(max_pixel / math.sqrt(mse))
    return psnr

# ESPCN

In [125]:
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

class ESPCN_model(nn.Module):
    def __init__(self, layer1_channel, layer2_channel, upscale_factor):
        super().__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(1, layer1_channel, kernel_size=5, stride=1, padding=2),
            nn.Tanh()
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(layer1_channel, layer2_channel, kernel_size=3, stride=1, padding=1),
            nn.Tanh()
        )

        self.layer3 = nn.Sequential(
            nn.ConvTranspose2d(layer2_channel, upscale_factor ** 2, kernel_size=3, stride=1, padding=1),
            nn.PixelShuffle(upscale_factor) # This function is literally built for ESPCN
        )

        self.apply(init_weights)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        return out

# UNet

In [7]:
# From PA2
class UNet(nn.Module):
    def __init__(self, kernel, num_filters, num_colours, num_in_channels):
        super().__init__()

        # Useful parameters
        stride = 2
        padding = kernel // 2
        output_padding = 1

        ############### YOUR CODE GOES HERE ############### 
        self.block1 = nn.Sequential(
            nn.Conv2d(num_in_channels, num_filters, kernel, stride, padding),
            nn.BatchNorm2d(num_filters),
            nn.ReLU()
        )
        self.block2 = nn.Sequential(
            nn.Conv2d(num_filters, 2 * num_filters, kernel, stride, padding),
            nn.BatchNorm2d(2 * num_filters),
            nn.ReLU()
        )
        self.block3 = nn.Sequential(
            nn.ConvTranspose2d(2 * num_filters, num_filters, kernel, stride, padding, output_padding, dilation=1),
            nn.BatchNorm2d(num_filters),
            nn.ReLU()
        )
        self.block4 = nn.Sequential(
            nn.ConvTranspose2d(num_filters + num_filters, num_colours, kernel, stride, padding, output_padding, dilation=1),
            nn.BatchNorm2d(num_colours),
            nn.ReLU()
        )
        self.output = nn.Conv2d(num_in_channels + num_colours, num_colours, kernel, padding=padding)
        ###################################################

    def forward(self, x):
        ############### YOUR CODE GOES HERE ###############
        a = self.block1(x)
        b = self.block2(a)
        c = self.block3(b)
        c1 = torch.cat((a, c), dim=1)
        d = self.block4(c1)
        d1 = torch.cat((x, d), dim=1)
        return self.output(d1)
        ###################################################

# Training

Code for validation

In [94]:
def run_validation_step(
    cnn,
    criterion,
    val_set,
):
    losses = []
    psnrs = []

    for i_batch, sample_batched in enumerate(val_set):
          input = sample_batched["y_low"]
          output = cnn(input)

          loss = criterion(output, sample_batched["y_high"])
          losses.append(loss.data.item())

          # Me trying to use psnr in tf
          # input_numpy = convert_to_tensor(input.squeeze().cpu().detach().numpy())
          # output_numpy = convert_to_tensor(output.squeeze().cpu().detach().numpy())

          # regular_upscale = Resize([args.resize, args.resize], InterpolationMode.BICUBIC)(input)

          psnrs.append(psnr(output, sample_batched["y_high"]))

    val_loss = np.mean(losses)
    val_psnr = np.mean(psnrs)
    return val_loss, val_psnr

Code for training (adapted from PA2)

In [127]:
def train(args, cnn=None):
    # Set the maximum number of threads to prevent crash in Teaching Labs
    # torch.set_num_threads(5)

    # Numpy random seed
    npr.seed(args.seed)

    # Save directory
    save_dir = "outputs/" + args.model

    # INPUT CHANNEL
    num_in_channels = 1

    # LOAD THE MODEL
    if cnn is None:
        cnn = ESPCN_model(64, 32, 3)

    # LOSS FUNCTION
    criterion = nn.MSELoss()

    if args.optimizor is "SGD":
      optimizer = torch.optim.SGD(cnn.parameters(), lr=1e-2, momentum=0.9, weight_decay=1e-4, nesterov=False)
    elif args.optimizor is "Adam":
      optimizer = torch.optim.Adam(cnn.parameters())

    # DATA
    print("Loading data...")

    if args.train_set is "ImageNet":
      trainset = torch.utils.data.DataLoader(ILSVRC2017Dataset(scale=args.scale, resize=args.resize), batch_size=args.batch_size)
    elif args.train_set is "BSDS":
      trainset = torch.utils.data.DataLoader(BSDS500Dataset(split="train", scale=args.scale, resize=args.resize), batch_size=args.batch_size)

    testset = torch.utils.data.DataLoader(BSDS500Dataset(split="val", scale=args.scale, resize=args.resize), batch_size=args.batch_size)

    # for item in trainset:
    #   print(item)

    # y.save("temp.jpg", "JPEG")

    # print("Transforming data...")
    # train_rgb, train_grey = process(x_train, y_train, downsize_input=args.downsize_input)
    # train_rgb_cat = get_rgb_cat(train_rgb, colours)
    # test_rgb, test_grey = process(x_test, y_test, downsize_input=args.downsize_input)
    # test_rgb_cat = get_rgb_cat(test_rgb, colours)

    # Create the outputs folder if not created already
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    print("Beginning training ...")
    if args.gpu:
        cnn.cuda()
    start = time.time()

    train_losses = []
    valid_losses = []
    valid_PSNR = []

    for epoch in range(args.epochs):
        # Train the Model
        cnn.train()  # Change model to 'train' mode
        losses = []

        for i_batch, sample_batched in enumerate(trainset):
          optimizer.zero_grad()

          input = sample_batched["y_low"]
          out = cnn(input)

          # print(out.size())
          # print(sample_batched["y_high"].size())

          loss = criterion(out, sample_batched["y_high"])
          loss.backward()
          optimizer.step()
          losses.append(loss.data.item())

        # plot training images
        avg_loss = np.mean(losses)
        train_losses.append(avg_loss)
        time_elapsed = time.time() - start
        print(
            "Epoch [%d/%d], Train Loss: %.4f, Time (s): %.2f"
            % (epoch + 1, args.epochs, avg_loss, time_elapsed)
        )

        # Evaluate the model
        # if epoch % 5 == 0:
        cnn.eval()  # Change model to 'eval' mode (BN uses moving mean/var).

        val_loss, val_psnr = run_validation_step(cnn, criterion, testset)

        time_elapsed = time.time() - start
        valid_losses.append(val_loss)
        valid_PSNR.append(val_psnr)

        print(
            "Epoch [%d/%d], Val Loss: %.4f, Val PSNR: %.4f, Time(s): %.2f"
            % (epoch + 1, args.epochs, val_loss, val_psnr, time_elapsed)
          )
        # else:
        #   valid_losses.append(None)

    # Plot training curve
    plot1 = plt.figure(1)
    plt.plot(train_losses, "ro-", label="Train")
    plt.plot(valid_losses, "go-", label="Validation")
    plt.legend()
    plt.title("Loss")
    plt.xlabel("Epochs")
    plt.savefig(save_dir + "/training_curve.png")

    plot2 = plt.figure(2)
    plt.plot(valid_PSNR, "yo-", label="PSNR")
    plt.legend()
    plt.title("PSNR")
    plt.xlabel("Epochs")
    plt.savefig(save_dir + "/PSNR_curve.png")

    if args.checkpoint:
        print("Saving model...")
        torch.save(cnn.state_dict(), args.checkpoint)

    return cnn

Train the model

In [ ]:
args = AttrDict()
args_dict = {
    "gpu": True,
    "valid": False,
    "checkpoint": "model.pkl",
    "model": "ESPCN",
    "scale": 3,
    "resize": 480,  # Images will be transformed to [resize, resize] square image
    "kernel": 3,
    "learn_rate": 0.001,
    "batch_size": 64,
    "epochs": 50,
    "seed": 42,
    "optimizor": "Adam",
    "train_set": "ImageNet"
}
args.update(args_dict)
cnn = train(args)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  This is separate from the ipykernel package so we can avoid doing imports until


Loading data...
Beginning training ...
Epoch [1/50], Train Loss: 0.0171, Time (s): 134.66
Epoch [1/50], Val Loss: 0.0049, Val PSNR: 23.1073, Time(s): 136.88
Epoch [2/50], Train Loss: 0.0053, Time (s): 271.66
Epoch [2/50], Val Loss: 0.0033, Val PSNR: 24.8678, Time(s): 273.91
Epoch [3/50], Train Loss: 0.0040, Time (s): 409.13
Epoch [3/50], Val Loss: 0.0028, Val PSNR: 25.5822, Time(s): 411.37


Visualization

In [ ]:
val_set = BSDS500Dataset(split="val", scale=3, resize=481)

index = 0
cnn.eval()
for item in val_set:

  if index == 50:
    img_arr = item["y_low"]
    img = ToPILImage()(img_arr)
    img.save("in.jpg", "JPEG")

    out = cnn(img_arr.unsqueeze(0))
    out_img = ToPILImage()(out.squeeze(0))
    out_img.save("out.jpg", "JPEG")
    
  index += 1


Result:


With regular upscale (using resize() function), the psnr is about 28.

Under setting batch_size = 64, epochs = 100:



In [ ]:
args = AttrDict()
args_dict = {
    "gpu": True,
    "valid": False,
    "checkpoint": "model-unet.pkl",
    "model": "UNet",
    "scale": 3,
    "resize": 480,  # Images will be transformed to [resize, resize] square image
    "kernel": 3,
    "learn_rate": 0.001,
    "batch_size": 64,
    "epochs": 100,
    "seed": 42,
    "optimizor": "Adam",
    "train_set": "ImageNet"
}
args.update(args_dict)
cnn = UNet(3, 32, 1, 1)
cnn = train(args, cnn)